# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

This data represents multiple marketing campaigns spanning a period of 5 years, from May 2008 to June 2013

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

In [3]:
df_campaign_outcome = pd.read_csv('sample_data/bank-additional-full.csv', sep = ';')

In [4]:
df_campaign_outcome.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
print(df_campaign_outcome.isnull().sum())

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


1.	While there are no missing values per se, several of the categorical variables have ‘unknown’ as a value, and those rows with any feature set to this value should be dropped.
2.	The feature named ‘duration’ should be dropped as stated in the problem statement.
3.	The datatype of the  categorical variables (job, education, marital, education,default, housing, loan, contact, month, day_of_week) should be converted to “category”
4.	The datatype of the following variables should be converted to integer: age, campaign, pdays, previous
5.	The datatype of the following variables should be converted to float: emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed


In [7]:
# Preprocessing steps
# Step 1: Replace 'unknown' with NaN and drop rows containing any NaN

import numpy as np
df_campaign_outcome.replace('unknown', np.nan, inplace=True)
df_campaign_outcome.dropna(inplace=True)

# Step 2: Drop the 'duration' column (not to be used in realistic modeling)
df_campaign_outcome.drop(columns=['duration'], inplace=True)

# Step 3: Convert all categorical columns to 'category' dtype
categorical_cols = [
    "job", "marital", "education", "default", "housing", "loan",
    "contact", "month", "day_of_week", "poutcome", "y"
]

for col in categorical_cols:
    df_campaign_outcome[col] = df_campaign_outcome[col].astype('category')

# Step 4: Convert month names to integers (e.g., 'jan' -> 1)
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
df_campaign_outcome['month'] = df_campaign_outcome['month'].map(month_mapping).astype(int)

# Step 5: Convert specific columns to int
df_campaign_outcome['age'] = df_campaign_outcome['age'].astype(int)
df_campaign_outcome['campaign'] = df_campaign_outcome['campaign'].astype(int)
df_campaign_outcome['pdays'] = df_campaign_outcome['pdays'].astype(int)
df_campaign_outcome['previous'] = df_campaign_outcome['previous'].astype(int)

# Step 6: Convert specific columns to float
df_campaign_outcome['emp.var.rate'] = df_campaign_outcome['emp.var.rate'].astype(float)
df_campaign_outcome['cons.price.idx'] = df_campaign_outcome['cons.price.idx'].astype(float)
df_campaign_outcome['cons.conf.idx'] = df_campaign_outcome['cons.conf.idx'].astype(float)
df_campaign_outcome['euribor3m'] = df_campaign_outcome['euribor3m'].astype(float)
df_campaign_outcome['nr.employed'] = df_campaign_outcome['nr.employed'].astype(float)

# Confirm changes
print("\nData types after preprocessing:")
print(df_campaign_outcome.dtypes)
print("\nShape after dropping rows with 'unknown':", df_campaign_outcome.shape)


Data types after preprocessing:
age                  int64
job               category
marital           category
education         category
default           category
housing           category
loan              category
contact           category
month                int64
day_of_week       category
campaign             int64
pdays                int64
previous             int64
poutcome          category
emp.var.rate       float64
cons.price.idx     float64
cons.conf.idx      float64
euribor3m          float64
nr.employed        float64
y                 category
dtype: object

Shape after dropping rows with 'unknown': (30488, 20)


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [8]:
df_campaign_outcome.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30488 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             30488 non-null  int64   
 1   job             30488 non-null  category
 2   marital         30488 non-null  category
 3   education       30488 non-null  category
 4   default         30488 non-null  category
 5   housing         30488 non-null  category
 6   loan            30488 non-null  category
 7   contact         30488 non-null  category
 8   month           30488 non-null  int64   
 9   day_of_week     30488 non-null  category
 10  campaign        30488 non-null  int64   
 11  pdays           30488 non-null  int64   
 12  previous        30488 non-null  int64   
 13  poutcome        30488 non-null  category
 14  emp.var.rate    30488 non-null  float64 
 15  cons.price.idx  30488 non-null  float64 
 16  cons.conf.idx   30488 non-null  float64 
 17  euribor3m       3

Business Objective: Given the input features which may be broadly classified into customer demographics, campaign contact details, and macro-economic variables, create a model to predict whether a given client will respond positively to the campaign and subscribe to the deposit.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# Use only first 15 features and target column
target = 'y'
features = df_campaign_outcome.columns[:15].tolist()

X = df_campaign_outcome[features]
y = df_campaign_outcome[target].apply(lambda v: 1 if v == 'yes' else 0)  # binary encode

# Define categorical and numerical columns
categorical_feats = X.select_dtypes(include='category').columns.tolist()
numerical_feats = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Define transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_feats),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_feats)
    ])


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [10]:

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [14]:
# Check for Class Imbalance in the Response variable by calculating percentage distribution of each outcome in the target variable 'y'
percentage_distribution = df_campaign_outcome['y'].value_counts(normalize=True) * 100

# Display formatted output
print("Percentage distribution of target variable 'y':")
print(percentage_distribution.round(2).astype(str) + '%')


Percentage distribution of target variable 'y':
y
no     87.34%
yes    12.66%
Name: proportion, dtype: object


Due to the highly imbalanced nature of the target variable (about 87% of all responses are "no"), a baseline performance model can be to **just always  predict the "majority class", which is "no"**. This should have an accuracy of atleast 85%.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [11]:
# Define models
models = {
    "Logistic Regression": LogisticRegression()

}


# Fit and compare models
print("\nModel Evaluation (Initial Scores):")
for name, model in models.items():
    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('clf', model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print(f"{name:20s} | ROC AUC: {auc:.4f}")


Model Evaluation (Initial Scores):
Logistic Regression  | ROC AUC: 0.7657
KNN                  | ROC AUC: 0.6956
Decision Tree        | ROC AUC: 0.6225
SVM                  | ROC AUC: 0.6981


### Problem 9: Score the Model

What is the accuracy of your model?

Logistic Regression  | ROC AUC: 0.7657

* Note: The choice of the evaluation metric as **ROC Area under the Curve** is driven by the fact that the response variable exhibits a **very high degree of class imbalance**. *italicized text*

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [16]:
# Fit and compare models
from sklearn.base import clone
from time import time

models = {
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(probability=True)
}

scores = []

for name, model in models.items():
    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('clf', clone(model))
    ])
    start_time = time()
    pipe.fit(X_train, y_train)
    end_time = time()
    train_acc = pipe.score(X_train, y_train)
    test_acc = pipe.score(X_test, y_test)
    fit_time = end_time - start_time

    scores.append({
        'Model': name,
        'Train Time': round(fit_time, 3),
        'Train Accuracy': round(train_acc, 4),
        'Test Accuracy': round(test_acc, 4)
    })

# Create DataFrame with model scores
df_scores = pd.DataFrame(scores)
print("\n=== Model Fit Times and Accuracies ===")
print(df_scores)


=== Model Fit Times and Accuracies ===
                 Model  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression       0.435          0.8874         0.8823
1                  KNN       0.133          0.9009         0.8742
2        Decision Tree       0.230          0.9929         0.8285
3                  SVM     345.400          0.8890         0.8834


Conclusion from Comparison on Test Data Accuracy:
------------------------------------------------
1. SVM looks most promising from an accuracy standpoint, with Logistic Regression coming in a close second.

2. However,the execution time for SVM is several orders of magnitude greater than the others (3056 times longer than KNN, 794 times longer than Logistic), making it a bad candidate for GridSearchCV unless care is taken to reduce dimensionality or the number of rows.   

3. Further more, the high class imbalance in the target variable 'y' is cause for pause, and to perhaps consider AUC instead, in order to focus on how well the model performs on the minority class of "yes".

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [13]:
models = {
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier()
    "SVM": SVC(probability=True)
}
param_grid = {
    "Logistic Regression": {'clf__C': [0.01, 0.1, 1, 10]},
    "KNN": {'clf__n_neighbors': [3, 5, 7]},
    "Decision Tree": {'clf__max_depth': [3, 5, 10]},
    "SVM": {'clf__C': [0.1, 1, 10], 'clf__kernel': ['linear', 'rbf']}
}

best_scores = {}
print("\nBest Hyperparameters via GridSearchCV:")
for name, model in models.items():
    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('clf', model)
    ])
    grid = GridSearchCV(pipe, param_grid[name], cv=5, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_scores[name] = (grid.best_score_, grid.best_params_)
    print(f"\n{name}:")
    print(f"Best ROC AUC: {grid.best_score_:.4f}")
    #print("Best Params:", grid.best_params_)
    print("Optimized Hyperparameters:")
    for param, value in grid.best_params_.items():
        print(f"  {param}: {value}")


# Report best model
best_model = max(best_scores.items(), key=lambda x: x[1][0])
print(f"\n🏆 Best Model: {best_model[0]} with ROC AUC = {best_model[1][0]:.4f}")



Best Hyperparameters via GridSearchCV:

Logistic Regression:
Best ROC AUC: 0.7796
Optimized Hyperparameters:
  clf__C: 0.01

KNN:
Best ROC AUC: 0.7279
Optimized Hyperparameters:
  clf__n_neighbors: 7

Decision Tree:
Best ROC AUC: 0.7894
Optimized Hyperparameters:
  clf__max_depth: 5

🏆 Best Model: Decision Tree with ROC AUC = 0.7894


##### Questions